# Bearbeiten und Cleanen von REM2030_v2015

In [1]:
import pandas as pd

In [2]:
# Lade die CSV-Datei
trips = pd.read_csv('REM2030_v2015.csv', delimiter=';', encoding='ISO-8859-1')

In [3]:
# Zeige die ersten 5 Zeilen an
print("Erste 5 Zeilen von REM2030_v2015:")
display(trips.head())

Erste 5 Zeilen von REM2030_v2015:


,id,deptyear,deptmonth,deptday,depthour,deptminute,arryear,arrmonth,arrday,arrhour,arrminute,distance,distance_to_company
0,1106000161,2011,6,16,8,7,2011,6,16,9,55,2.66,0.03
1,1106000161,2011,6,16,10,32,2011,6,16,10,34,3.19,2.28
2,1106000161,2011,6,16,15,15,2011,6,16,15,17,0.63,2.24
3,1106000161,2011,6,17,11,47,2011,6,17,11,51,1.70,0.70
4,1106000161,2011,6,17,11,51,2011,6,17,11,52,1.54,1.18


In [4]:
# Berechnung der Reisezeit 'travel_time' und der Dauer der Aufzeichnung 'days_duration'
trips['departure'] = pd.to_datetime(
    trips[['deptyear', 'deptmonth', 'deptday', 'depthour', 'deptminute']]
    .rename(columns={'deptyear': 'year', 'deptmonth': 'month', 'deptday': 'day', 'depthour': 'hour', 'deptminute': 'minute'})
)
trips['arrival'] = pd.to_datetime(
    trips[['arryear', 'arrmonth', 'arrday', 'arrhour', 'arrminute']]
    .rename(columns={'arryear': 'year', 'arrmonth': 'month', 'arrday': 'day', 'arrhour': 'hour', 'arrminute': 'minute'})
)

# Reisezeit in Minuten berechnen und Zeilen mit weniger als 1 Minute entfernen
trips['travel_time'] = (trips['arrival'] - trips['departure']).dt.total_seconds() / 60.0
trips = trips[trips['travel_time'] >= 1]

## Dauer der Aufzeichnung in Tagen (mindestens 1 Tag)
trips['days_duration'] = (trips['arrival'] - trips['departure']).dt.ceil('D').dt.days

# Berechnung der durchschnittlichen Geschwindigkeit (auf 2 Nachkommastellen)
trips['avg_speed'] = trips.apply(
    lambda row: round(row['distance'] / (row['travel_time'] / 60), 2) if row['travel_time'] > 0 else None,
    axis=1
)

In [5]:
# Kalenderwoche und Wochentag aus dem Abfahrtsdatum extrahieren
trips['calendar_week'] = trips['departure'].dt.isocalendar().week
trips['week_day'] = trips['departure'].dt.day_name()

# Anzeigen der ersten Zeilen zur Überprüfung
print("Erste Zeilen mit Kalenderwoche und Wochentag:")
display(trips[['departure', 'calendar_week', 'week_day']].head())

Erste Zeilen mit Kalenderwoche und Wochentag:


,departure,calendar_week,week_day
0,2011-06-16 08:07:00,24,Thursday
1,2011-06-16 10:32:00,24,Thursday
2,2011-06-16 15:15:00,24,Thursday
3,2011-06-17 11:47:00,24,Friday
4,2011-06-17 11:51:00,24,Friday


In [6]:
# Sicherstellen, dass alle relevanten Spalten numerisch sind
numeric_columns = ['travel_time', 'days_duration', 'avg_speed', 'distance', 'distance_to_company']
for column in numeric_columns:
    trips[column] = pd.to_numeric(trips[column], errors='coerce').fillna(0)
    print(f"Spalte '{column}' erfolgreich in numerische Werte umgewandelt.")

# Überprüfen auf fehlende Werte
missing_values = trips.isnull().sum()
if missing_values.any():
    print("\nFehlende Werte gefunden:")
    print(missing_values)
else:
    print("\nKeine fehlenden Werte gefunden.")

# Überprüfen, ob keiner der Werte 0 ist (außer 'distance_to_company')
critical_columns = [col for col in numeric_columns if col != 'distance_to_company']
zero_values = (trips[critical_columns] == 0).any()
if zero_values.any():
    print("\nWarnung: Es gibt Spalten mit einem Wert von 0:")
    print(zero_values[zero_values])
else:
    print("\nAlle Werte sind von Null verschieden (ausgenommen distance_to_company).")

Spalte 'travel_time' erfolgreich in numerische Werte umgewandelt.
Spalte 'days_duration' erfolgreich in numerische Werte umgewandelt.
Spalte 'avg_speed' erfolgreich in numerische Werte umgewandelt.
Spalte 'distance' erfolgreich in numerische Werte umgewandelt.
Spalte 'distance_to_company' erfolgreich in numerische Werte umgewandelt.

Keine fehlenden Werte gefunden.

Alle Werte sind von Null verschieden (ausgenommen distance_to_company).


In [7]:
# Ergebnis anzeigen
print("\nErste 5 Zeilen nach den Berechnungen:")
display(trips.head())


Erste 5 Zeilen nach den Berechnungen:


,id,deptyear,deptmonth,deptday,depthour,deptminute,arryear,arrmonth,arrday,arrhour,arrminute,distance,distance_to_company,departure,arrival,travel_time,days_duration,avg_speed,calendar_week,week_day
0,1106000161,2011,6,16,8,7,2011,6,16,9,55,2.66,0.03,2011-06-16 08:07:00,2011-06-16 09:55:00,108.0,1,1.48,24,Thursday
1,1106000161,2011,6,16,10,32,2011,6,16,10,34,3.19,2.28,2011-06-16 10:32:00,2011-06-16 10:34:00,2.0,1,95.70,24,Thursday
2,1106000161,2011,6,16,15,15,2011,6,16,15,17,0.63,2.24,2011-06-16 15:15:00,2011-06-16 15:17:00,2.0,1,18.90,24,Thursday
3,1106000161,2011,6,17,11,47,2011,6,17,11,51,1.70,0.70,2011-06-17 11:47:00,2011-06-17 11:51:00,4.0,1,25.50,24,Friday
4,1106000161,2011,6,17,11,51,2011,6,17,11,52,1.54,1.18,2011-06-17 11:51:00,2011-06-17 11:52:00,1.0,1,92.40,24,Friday


In [8]:
### KONTROLLE DER GESCHWINDIGKEITEN
### TEST, KANN DANACH GELÖSCHT WERDEN
filtered_trips = trips[(trips['id'] == 1106000161) & (trips['arrival'] < '2011-07-02 00:00:00')]
display(filtered_trips[['departure', 'arrival', 'distance', 'travel_time', 'avg_speed']])

,departure,arrival,distance,travel_time,avg_speed
0,2011-06-16 08:07:00,2011-06-16 09:55:00,2.66,108.0,1.48
1,2011-06-16 10:32:00,2011-06-16 10:34:00,3.19,2.0,95.70
2,2011-06-16 15:15:00,2011-06-16 15:17:00,0.63,2.0,18.90
3,2011-06-17 11:47:00,2011-06-17 11:51:00,1.70,4.0,25.50
4,2011-06-17 11:51:00,2011-06-17 11:52:00,1.54,1.0,92.40
5,2011-06-17 12:17:00,2011-06-17 12:20:00,3.41,3.0,68.20
6,2011-06-20 10:10:00,2011-06-20 10:37:00,3.17,27.0,7.04
7,2011-06-20 13:15:00,2011-06-20 13:43:00,18.14,28.0,38.87
8,2011-06-21 09:43:00,2011-06-21 09:47:00,2.65,4.0,39.75
9,2011-06-21 10:46:00,2011-06-21 11:15:00,9.88,29.0,20.44


In [9]:
# Die geänderten Daten in einer neuen Datei speichern
trips.to_csv('REM2030_cleaned.csv', index=False)